<a href="https://colab.research.google.com/github/Shreshta001/ML_DL_AI_vs_REAL/blob/main/cifakedatasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision transformers diffusers


#Load the CIFAR-10 Dataset

In [ ]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Define transforms for preprocessing the CIFAR-10 images
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resizing images to 32x32 pixels
    transforms.ToTensor(),        # Convert images to PyTorch tensors
])

# Download and load the CIFAR-10 training dataset
cifar10_train = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transform
)

# Create a DataLoader for easy batch handling
train_loader = DataLoader(cifar10_train, batch_size=64, shuffle=True)

# Check the number of training samples
print(f'Total training images: {len(cifar10_train)}')


100%|██████████| 170498071/170498071 [00:03<00:00, 44557055.91it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Total training images: 50000


In [ ]:
from diffusers import StableDiffusionPipeline
import torch
import os

# Initialize the Stable Diffusion model (adjust for other LDMs if needed)
model_id = "CompVis/stable-diffusion-v1-4"
device = "cuda" if torch.cuda.is_available() else "cpu"

pipeline = StableDiffusionPipeline.from_pretrained(model_id)
pipeline.to(device)

# Function to generate synthetic images
def generate_synthetic_images(prompt, num_images=1000, save_path="./cifake"):
    os.makedirs(save_path, exist_ok=True)

    for i in range(num_images):
        # Generate an image from the text prompt
        image = pipeline(prompt).images[0]

        # Save the generated image
        image_path = os.path.join(save_path, f"synthetic_{i}.png")
        image.save(image_path)
        print(f"Saved {image_path}")

# Generate synthetic images with a text prompt
generate_synthetic_images("a photo of a cat", num_images=1000)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

...

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

Saved ./cifake/synthetic_0.png


  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
import os
from PIL import Image
from torchvision.datasets import ImageFolder

# Define paths for CIFAR-10 and synthetic images
cifar10_path = "./data/cifar10"
synthetic_path = "./cifake"

# Combine the real and synthetic images into a single dataset
class CIFAKE(torchvision.datasets.VisionDataset):
    def __init__(self, real_dir, synthetic_dir, transform=None):
        self.real_images = ImageFolder(real_dir, transform=transform)
        self.synthetic_images = ImageFolder(synthetic_dir, transform=transform)
        self.transform = transform

        # Combine real and synthetic images
        self.data = self.real_images.samples + self.synthetic_images.samples
        self.targets = [0] * len(self.real_images) + [1] * len(self.synthetic_images)  # 0 for real, 1 for synthetic

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path, _ = self.data[idx]
        image = Image.open(image_path)
        label = self.targets[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Initialize the CIFAKE dataset
cifake_dataset = CIFAKE(real_dir=cifar10_path, synthetic_dir=synthetic_path, transform=transform)

# Create a DataLoader for the CIFAKE dataset
cifake_loader = DataLoader(cifake_dataset, batch_size=64, shuffle=True)

print(f"Total images in CIFAKE dataset: {len(cifake_dataset)}")
